In [1]:
%matplotlib widget
import os, sys
import numpy as np

In [2]:
f = 7471
x_c = 19.5
y_c = -6
width = 7952
height = 5304
k1 = 0.09
k2 = 0.04629766820693931
k3 = -0.07699880304345112
t1 = 0.00043476760052012665
t2 = 0.00023383541355524783

In [3]:
def image2pixel(p, f, x_c, y_c, width, height):
    # output: coordinates in the image pixels crs which origin is upper left corner (cv standard axis directions)
    K = np.array([[f, 0, x_c + width/2.0], [0, -f, height/2.0 - y_c]])
    p_n = K.dot([p[0], p[1], 1])
    return True, p_n

def distort(p, f, x_c, y_c, k1, k2, k3, t1, t2):
    # p is the image point's normalized coordinates
    # f is the focal length
    # x_c, y_c are the principle point from the image center
    
    r2 = p[0] * p[0] + p[1] * p[1]    
    rp = 1 + k1*r2 + k2*r2*r2 + k3*r2*r2*r2

    rx = p[0] * rp
    ry = p[1] * rp

    tx = t2 * (r2 + 2 * p[0] * p[0]) - 2 * t1 * p[0] * p[1];
    ty = - t1 * (r2 + 2 * p[1] * p[1]) + 2 * t2 * p[0] * p[1];
    
    x = rx + tx;
    y = ry + ty;
    
    return True, np.array([x, y])


In [4]:
p_gt = [0.234, -0.195]

# note: this model defines the radial and tangential distortions with normalized coordinates
distort_fn = lambda p: distort(p, f, x_c, y_c, k1, k2, k3, t1, t2)
result, p_d = distort_fn(p_gt)
result, p_d = image2pixel(p_d, f, x_c, y_c, width, height)
print(p_d)

[5759.55115266 4128.20892124]


In [5]:
def pixel2image(p_d, x_c, y_c, width, height):
    K = np.array([[f, 0, x_c + width/2.0], [0, -f, height/2.0 - y_c], [0, 0, 1]])
    p_u = np.linalg.inv(K).dot([p_d[0], p_d[1], 1])
    return True, p_u[:2]

def undistort(p, distort_fn, epsilon=1e-10, max_iter_num=10, verbose=False):
    
    p_u = p.copy()
    
    result, p_d_i = distort_fn(p_u);
    if result == False:
        print('Distortion is unsucessful!')
        return False, np.one((2, 1)) * np.nan
    delta_p = p_d_i - p_u
    
    epsilon = 1e-10
    iter_num = 0
    while np.max(np.abs(p_u - (p - delta_p))) > epsilon and iter_num < max_iter_num:
        p_u = p - delta_p
        result, p_d_i = distort_fn(p_u)
        if result == False:
            print('Distortion is unsucessful!')
            return False, np.one((2, 1)) * np.nan
        delta_p = p_d_i - p_u
        iter_num += 1
        
        if verbose:
            num2str = lambda x: str(round(x, 10))
            print('#' + num2str(iter_num) + " x=" + num2str(p_u[0]) + 
                  " y=" + num2str(p_u[0]) + " delta=" + num2str(np.max(np.abs(p_u - (p - delta_p)))))
    
    if iter_num == max_iter_num:
        print('Did not converge!')
        return False, p_u
    
    return True, p_u

In [6]:
result, p_c = pixel2image(p_d, x_c, y_c, width, height)
result, p_u = undistort(p_c, distort_fn, verbose=True)

#1 x=0.2339412609 y=0.2339412609 delta=6.03512e-05
#2 x=0.234001612 y=0.234001612 delta=1.6563e-06
#3 x=0.2339999557 y=0.2339999557 delta=4.55e-08
#4 x=0.2340000012 y=0.2340000012 delta=1.2e-09
#5 x=0.234 y=0.234 delta=0.0
